In [60]:
import matplotlib.pyplot as plt
from matplotlib import colors
import numpy as np
from stompy.grid import unstructured_grid
import glob
import six
import os
from stompy import utils
import xarray as xr
%matplotlib notebook


In [35]:
import postprocess_v00 as post
six.moves.reload_module(post)

<module 'postprocess_v00' from '/home/rusty/src/microplastic_sfbay/postprocess/postprocess_v00.py'>

In [29]:
grid_fn="/opt2/sfb_ocean/suntans/runs/merged_018_20171227/ptm_average.nc_0000.nc"  
grid=post.grid_from_ptm_hydro(grid_fn)
M=grid.smooth_matrix(f=0.5,dx='grid',A='grid',V='grid',K='scaled')

INFO:utils:46602/99089
INFO:utils:90713/99089


In [31]:
def smooth(c): 
    for _ in range(20):
        c=M.dot(c)
    return c


In [58]:
class BayConcFigure(object):
    figsize=(9,7)
    vmin=1e-6
    vmax=1e-2
    zoom=(517521., 613202., 4139744., 4230105.)
    cax_loc=[0.7,0.25,0.03,0.35]
    txt_loc=[0.65,0.7] # in ax coords
    cbar_label="Particles/m$^2$"
    num=None
    def __init__(self,ds,conc=None,**kw):
        utils.set_keywords(self,kw)
        self.ds=ds
        if conc is None:
            conc=self.ds['conc'].values
                        
        self.fig=plt.figure(figsize=(10,8),num=self.num)
        self.ax=self.fig.add_subplot(1,1,1)
        
        self.ccoll=grid.plot_cells(values=conc.clip(self.vmin,self.vmax),
                                   cmap='jet',norm=colors.LogNorm(vmin=self.vmin,vmax=self.vmax),
                                   edgecolor='face',lw=0.4,ax=self.ax)
        self.cax=self.fig.add_axes(self.cax_loc)
        plt.colorbar(self.ccoll,cax=self.cax,label=self.cbar_label)
        self.ax.set_aspect('equal')
        self.ax.xaxis.set_visible(0)
        self.ax.yaxis.set_visible(0)
        self.ax.axis(self.zoom)
        self.fig.subplots_adjust(left=0.01,right=0.99,top=0.99,bottom=0.01)
        
        self.add_labels()
    def __del__(self):
        self.ds.close()
    def add_labels(self):
        texts=[self.behavior_label()]
        texts.append(self.average_label())
        self.ax.text(self.txt_loc[0],self.txt_loc[1],"\n".join(texts),
                     fontsize=14,va='top',ha='left',transform=self.ax.transAxes)
    def behavior_label(self):
        # go from a list of groups to a label
        behaviors=[ post.PtmRun.group_to_src_behavior(group)[1]
                    for group in self.ds.ptm_groups.values ]
        behaviors.sort()
        assert behaviors[0]==behaviors[-1]
        behavior=behaviors[0]
        if behavior=='none':
            label='Passive'
        elif behavior.startswith('up'):
            w_mmps=float(behavior.replace('up',''))/1000.0
            label=f'Rise {w_mmps:.1f} mm/s'
        elif behavior.startswith('down'):
            w_mmps=float(behavior.replace('down',''))/1000.0
            label=f'Sink {w_mmps:.1f} mm/s'
        return label
    def average_label(self):
        if np.all(self.ds.z_range.values==[0,0.5]):
            return "Near bed"
        elif np.all(self.ds.z_range.values==[-0.5,0.0]):
            return "Near surface"
        elif np.all(self.ds.z_range.values==[0,0]):
            return "Full depth"
        else:
            return "Vertical range: %.2f – %.2f"%(self.ds.z_range.values[0],
                                                  self.ds.z_range.values[1])

In [ ]:
TODO: add date range to each.

In [ ]:
plt.ioff()
for ptm_conc in glob.glob('processed/201*_201*/[0-9]*/particles*-v01.nc'):
    print(ptm_conc)
    img_fn=ptm_conc.replace('.nc','.png')
    if os.path.exists(img_fn): continue

    ds=xr.open_dataset(ptm_conc)
    bf=BayConcFigure(ds,conc=smooth(ds.conc.values))
    bf.fig.savefig(img_fn,dpi=150)
plt.ion()

processed/20171130_20171215/0.005/particles-bed-15days-v01.nc
processed/20171130_20171215/0.005/particles-surf-15days-v01.nc
processed/20171130_20171215/0.005/particles-avg-15days-v01.nc
processed/20171130_20171215/0.0005/particles-bed-15days-v01.nc
processed/20171130_20171215/0.0005/particles-surf-15days-v01.nc
processed/20171130_20171215/0.0005/particles-avg-15days-v01.nc
processed/20171130_20171215/0.05/particles-bed-15days-v01.nc
processed/20171130_20171215/0.05/particles-surf-15days-v01.nc
processed/20171130_20171215/0.05/particles-avg-15days-v01.nc
processed/20171130_20171215/0.0/particles-bed-15days-v01.nc
processed/20171130_20171215/0.0/particles-surf-15days-v01.nc
processed/20171130_20171215/0.0/particles-avg-15days-v01.nc
processed/20171030_20171114/0.005/particles-bed-15days-v01.nc
processed/20171030_20171114/0.005/particles-surf-15days-v01.nc
processed/20171030_20171114/0.005/particles-avg-15days-v01.nc
processed/20171030_20171114/0.0005/particles-bed-15days-v01.nc
processe

/opt/anaconda3/lib/python3.7/site-packages/matplotlib/pyplot.py:514: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


processed/20171030_20171114/0.0/particles-bed-15days-v01.nc


/opt/anaconda3/lib/python3.7/site-packages/matplotlib/pyplot.py:514: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


processed/20171030_20171114/0.0/particles-surf-15days-v01.nc


...

/opt/anaconda3/lib/python3.7/site-packages/matplotlib/pyplot.py:514: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


processed/20170630_20170715/0.05/particles-bed-15days-v01.nc


/opt/anaconda3/lib/python3.7/site-packages/matplotlib/pyplot.py:514: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


processed/20170630_20170715/0.05/particles-surf-15days-v01.nc


/opt/anaconda3/lib/python3.7/site-packages/matplotlib/pyplot.py:514: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


In [53]:
ds.z_range

<xarray.DataArray 'z_range' (two: 2)>
array([0. , 0.5])
Dimensions without coordinates: two